In [1]:
# Import all required libraries
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers import AutoModel
import torch
import spacy
import spacy_transformers  # Explicitly import spacy_transformers
from pathlib import Path
import re



print("All imports successful!")

c:\Users\yby39\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful!


In [3]:
path=Path(r"./model-best")
# Load the best trained model
nlp = spacy.load(path)

In [4]:

model_name = "manishiitg/resume-ner"  # This is trained for NER
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

print(f"Model loaded: {model_name}")
print(f"Model max length: {tokenizer.model_max_length}")
print(f"Available labels: {list(model.config.id2label.values())}")

Model loaded: manishiitg/resume-ner
Model max length: 512
Available labels: ['CARDINAL', 'GPE', 'Phone', 'Skills', 'ORG', 'EducationDegree', 'ExperianceYears', 'DATE', 'PERSON', 'Designation', 'O']


In [5]:
text="Ram Edupuganti - Resume Current: Software Development Director - Oracle Inc  - Email me on Indeed: indeed.com/r/Ram-Edupuganti/3ecdecbcba549e21  \u2022 Offering over 22 years of rich national & international experience in developing & deploying software appplicatoins development, customized solutions across Oracle Fusion HCM Global Core HR, Global Payroll, Global Absences, Compensation and all Talent modules - Performance Management, Goals Management, Career Development, Succession Management & Talent Review Meeting \u2022 Directed planning, strategy development and implementation & business solution delivery with recent successful implantations at Schneider Electric, Macy's, Pike International, British Telcom. \u2022 Accomplished in relational data modeling, data warehouse design and implementation, object oriented design and development, developing transactional & business analytics in CRM, SCM and HCM domains from client server to cloud SaaS. \u2022 Led large-scale business application architecture and design efforts; developed and maintained solutions for various business functional areas; assisted in resolving integration and interface issues between various systems with focus on optimizing performance and scalability \u2022 Extensive experience in all aspects of project management including budgeting and cost optimization, risk assessments and control, technical feasibility studies, project scope definition, estimations & cost control & so on \u2022 Recruit, Develop and led high-performing teams with high motivation and enhanced cross- functional collaboration  WORK EXPERIENCE  Software Development Director  Oracle Inc -  April 2015 to Present  Employer: Oracle Inc -  January 1998 to Present  98 with Oracle Inc. Growth Path & Deputations:  Role: Senior Development Manager  Employer: Oracle Inc -  December 2011 to March 2015  Senior Development Manager  Employer: Oracle Inc -  March 2010 to November 2011  https://www.indeed.com/r/Ram-Edupuganti/3ecdecbcba549e21?isid=rex-download&ikw=download-top&co=IN   Role: Principal Applications Developer  Oracle Inc -  September 2004 to February 2010  Project Lead  Oracle Inc -  September 1999 to August 2004  Senior Software Developer  Oracle Inc, Inc. -  January 1998 to August 1999  R-Systems -  Sacramento, CA -  January 1997 to January 1998  Key Result Areas: \u2022 Providing overall leadership to the entire project team, mapping clients' requirements, transforming requirements into stipulations and providing them best solutions within the scope of project requirements \u2022 Creating and managing the estimates, project plan, project schedule, resource allocation and expenses; monitoring and reporting on standards & performance targets \u2022 Ensuring the delivery of quality releases on schedules using continuous improvement initiatives; adhering to quality norms throughout the project implementation \u2022 Working on service strategy, transition, operations, process improvement, process management, team building, training, hiring and client relationship management \u2022 Identifying and implementing strategies for building team effectiveness by promoting a spirit of cooperation among the team members \u2022 Supporting continuous improvement by investigating alternatives and new technologies and presenting the same for architectural review \u2022 Liaising with stakeholders during the course of problem diagnoses, requirements gathering, detailed level design, development, system test and production implementation to ensure that optimal resolutions are achieved  KNOWLEDGE PURVIEW \u2022 Machine Learning Algorithms supervised, unsupervised, NLP, chat bot and deep learning with Python \u2022 Building transactional applications & analytical solutions in SaaS model \u2022 Oracle fusion role based data security set up and customization for complex security requirements \u2022 Reporting Layer (Subject Areas) for fusion HCM cloud for all HCM modules \u2022 Data Warehousing design methodologies, star and snowflake schema designs, aggregations \u2022 OBIEE, Data Visualization Desktop, BICS, DVCS, Oracle Analytics Cloud (OAC), Tableau \u2022 Design and build dashboards, KPIs using Oracle BI platform and OTBI, Standard Reports using BI Publisher \u2022 Design and build mappings, knowledge modules, load plans using Oracle Data Integrator (ODI)    \u2022 Relational data modeling, Object Oriented Modeling & Design, UML, SOAP \u2022 Oracle 10g, 11g, 12c \u2022 SQL, PL/SQL, HTML, XML, JAVA, JDK, J2EE, Oracle ADF, Oracle JDeveloper \u2022 Applications Development for Oracle CRM, Oracle ebusiness SCM, \u2022 Applications/database/SQL/Batch program performance tuning \u2022 UNIX, Windows, Linux  R-Systems -  April 1996 to January 1998  Growth Path & Deputations:  Department of Corrections -  Sacramento, CA -  February 1996 to January 1997  Secretary of State  Salem, Tamil Nadu -  December 1995 to February 1996  Oracle Corporation -  Redwood Shores, CA -  January 1995 to December 1995  EDUCATION  M.S. in Computer Engineering with honors  Texas A&M University -  Kingsville, TX  1994  B.S. in Electronics & Communications Engineering with honors  Nagarjuna University  1992  SKILLS  SOFTWARE DEVELOPMENT (3 years), STRUCTURED SOFTWARE (3 years), BUSINESS INTELLIGENCE (1 year), ORACLE (1 year), ARCHITECTURE (Less than 1 year)  ADDITIONAL INFORMATION  CORE COMPETENCIES  Software Development & Consulting  Oracle Fusion Applications Architecture and Functionality    Oracle Business Intelligence development and implementations  Technology Planning  Delivery Management  Client Engagements (Stakeholders/Business)  Continuous Process Enhancement  Agile/Scrum Methodologies  Project Management"


In [6]:
cleaned_text = re.sub(r'[^\x00-\x7F]+', '', text)
text = cleaned_text.replace('\n',' ').replace('\r','')
text = re.sub('\s+',' ',text).strip()
text[:100]

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\yby39\AppData\Local\Temp\ipykernel_31532\2532585395.py:3: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+',' ',text).strip()


'Ram Edupuganti - Resume Current: Software Development Director - Oracle Inc - Email me on Indeed: in'

In [7]:
L = {}
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)
    if ent.label_ !='Skills':
        L[ent.label_] = L.get(ent.label_, []) + [ent.text]
    else:
        skills = ent.text.split(':')
        for i in range(len(skills)):
            skills2 = skills[i].split(',')
            for i in range(len(skills2)):
                skills2[i] = skills2[i].strip()
                if len(skills2[i]) < 30:
                    L['Skills'] = L.get('Skills', []) + [skills2[i]]


print(L)

Ram Edupuganti Name
Software Development Director Designation
Oracle Inc Companies worked at
Oracle Companies worked at
Software Development Director Designation
Oracle Companies worked at
Senior Development Manager Designation
Oracle Companies worked at
Senior Development Manager Designation
Oracle Companies worked at
Principal Applications Developer Designation
Project Lead Designation
Senior Software Developer Designation
SQL, PL/SQL, HTML, XML, JAVA, JDK, J2EE, Oracle ADF, Oracle JDeveloper Applications Development for Oracle CRM, Oracle ebusiness SCM, Applications/database/SQL/Batch program performance tuning UNIX, Windows, Linux R-Systems - April 1996 to January 1998 Growth Path & Deputations: Department of Corrections - Sacramento, CA - February 1996 to January 1997 Secretary of State Salem, Tamil Nadu - December 1995 to February 1996 Oracle Corporation - Redwood Shores, CA - January 1995 to December 1995 EDUCATION M.S. in Computer Engineering Skills
Texas A&M University College

In [8]:
inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    is_split_into_words=False
)

# 2. Forward pass
with torch.no_grad():
    outputs = model(**inputs)          # outputs.logits shape: [batch, seq_len, num_labels]

# 3. Take the most probable label for each token
pred_ids = torch.argmax(outputs.logits, dim=2)[0]    # shape: [seq_len]

# 4. Convert back to words + labels
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
labels = [model.config.id2label[id_.item()] for id_ in pred_ids]



j=-1
for i, (token, label) in enumerate(zip(tokens, labels)):
    if(j>i):
        continue
    if label != "O":
        # Start a span with the current token
        span = token
        j = i + 1
        while j < len(labels) and labels[j] == label:# Check consecutive tokens with the same label
            span +=" "
            span += tokenizer.convert_tokens_to_string([tokens[j]])# Concatenate subsequent tokens with the same label
            j += 1
    
        
            print(f"{span:<30} {label}")

software development           Designation
software development director  Designation
oracle inc                     ORG
22 years                       ExperianceYears
april 2015                     DATE
january 1998                   DATE
present 98                     DATE
december 2011                  DATE
march 2015                     DATE
senior development             Designation
senior development manager     Designation
march 2010                     DATE
november 2011                  DATE
september 2004                 DATE
february 2010                  DATE
september 1999                 DATE
august 2004                    DATE
january 1998                   DATE
august 1999                    DATE
january 1997                   DATE
january 1998                   DATE


In [9]:
email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
skills_pattern=r"\b(Python|Java|C\+\+|C#|JavaScript|TypeScript|Go|Rust|Ruby|PHP|Swift|Kotlin|R|MATLAB|SQL|NoSQL|MongoDB|PostgreSQL|MySQL|HTML|CSS|React|Angular|Vue|Django|Flask|Spring|Node\.js|Express|TensorFlow|PyTorch|Keras|Scikit-learn|Pandas|NumPy|Docker|Kubernetes|Git|AWS|Azure|GCP|Linux|Unix|Bash|Shell)\b"
experience_pattern=r"(\d+)\s+(years|year)"
skills = re.findall(skills_pattern, text)
print(f"Skills found: {skills}")
emails = re.findall(email_pattern, text)
print(f"Emails found: {emails}")
if emails:
    L['Email Address'] = L.get('EMAIL', []) + emails
if skills:
    L['Skills'] = L.get('Skills', []) + skills

Skills found: ['R', 'Python', 'SQL', 'SQL', 'HTML', 'SQL', 'Linux', 'R']
Emails found: []


In [10]:
degree_pattern = r"""
\b(
    # Common abbreviated forms
    (?:B.S.\s\.?\s?[A-Z][a-zA-Z]*|M\.?\s?[A-Z][a-zA-Z]*|Ph\.?D\.?|MBA|LL\.?B\.?|LL\.?M\.?) |
    
    # Bachelor's, Master's, Doctorate, Associate's
    (?:Bachelor(?:’s|\'s)?\s(?:degree\s)?(?:in\s[\w\s,&/()-]+(?:\s(?:or|and)\s[\w\s,&/()-]+)*)?) |
    (?:Master(?:’s|\'s)?\s(?:degree\s)?(?:in\s[\w\s,&/()-]+(?:\s(?:or|and)\s[\w\s,&/()-]+)*)?) |
    (?:Doctor(?:ate| of Philosophy| of [\w\s,&/()-]+)) |
    (?:Associate(?:’s|\'s)?\s(?:degree\s)?(?:in\s[\w\s,&/()-]+(?:\s(?:or|and)\s[\w\s,&/()-]+)*)?) |
    
    # Diploma or Certification programs
    (?:Diploma\s(?:in\s[\w\s,&/()-]+)?) |
    (?:Certificate\s(?:in\s[\w\s,&/()-]+)?) |
    
    # Generic Degree + field
    (?:Degree\s(?:in\s[\w\s,&/()-]+)?)
)\b
"""

d_pattern = re.compile(degree_pattern, re.IGNORECASE | re.VERBOSE)
degree= re.findall(degree_pattern, text)
print(f"Degrees found: {degree}")

Degrees found: []


In [11]:
for key in L:
    L[key] = list(set(L[key]))  # Remove duplicates

for key in L:
    print(f"{key}: {L[key]}")

Name: ['Ram Edupuganti']
Designation: ['Principal Applications Developer', 'Senior Software Developer', 'Software Development Director', 'Senior Development Manager', 'Project Lead']
Companies worked at: ['Oracle', 'Oracle Inc']
Skills: ['PL/SQL', 'XML', 'Windows', 'ORACLE (1 year)', 'Oracle ebusiness SCM', 'Linux', 'Python', 'DEVELOPMENT (3 years)', 'SQL', 'J2EE', 'JAVA', 'Oracle ADF', 'STRUCTURED SOFTWARE (3 years)', 'HTML', 'JDK', 'R']
College Name: ['Texas A&M University', 'Nagarjuna University']
Degree: ['B.S. in Electronics & Communications Engineering']


In [12]:
job = "Job Title: Software Engineer About the Role We are seeking a skilled and motivated Software Engineer to join our development team. You will be responsible for designing, developing, testing, and maintaining high-quality software solutions. The ideal candidate has a solid foundation in computer science, enjoys solving complex problems, and is eager to collaborate in a fast-paced, agile environment. Responsibilities - Design, develop, and maintain scalable and reliable software applications. - Write clean, efficient, and well-documented code. - Collaborate with product managers, designers, and other engineers to deliver new features. - Participate in code reviews, testing, and debugging. - Ensure software performance, security, and maintainability. - Contribute to continuous integration and deployment (CI/CD) processes. - Stay updated with emerging technologies and industry best practices. Requirements - Bachelor’s degree in Computer Science, Software Engineering, or related field (or equivalent experience). - Strong proficiency in one or more programming languages (e.g., Python, Java, C++, or JavaScript). - Experience with databases (SQL/NoSQL) and API development. - Familiarity with version control systems (e.g., Git). - Understanding of software development methodologies (Agile/Scrum). - Strong problem-solving and analytical skills. - Excellent communication and teamwork abilities. Preferred Qualifications - Experience with cloud platforms (AWS, Azure, or GCP). - Knowledge of DevOps tools and CI/CD pipelines. - Familiarity with front-end frameworks (React, Angular, or Vue). - Exposure to containerization technologies (Docker, Kubernetes). What We Offer - Competitive salary and benefits package. - Flexible working arrangements (remote/hybrid). - Professional development and training opportunities. - A collaborative and innovative work environment."

# Job description parsing 

## HuggingFace model for job description NER

In [13]:
job="A highly accomplished software engineering leader with over 10 years of experience in software design, development, and project management across enterprise-level environments. Demonstrated success in leading cross-functional teams, overseeing the full software development lifecycle, and delivering scalable, high-performance applications. Skilled in a wide range of technologies including Java, Python, SQL, XML, J2EE, Linux, and enterprise solutions, with expertise in both back-end and front-end development. Adept at database design, systems integration, and structured software methodologies, with proven ability to align technology strategies with business goals. Experienced in working with global organizations, fostering innovation, and mentoring engineering teams to achieve excellence. Strong academic background with a Bachelor of Science in Electronics & Communications Engineering, bringing both technical depth and leadership expertise. Recognized for driving efficiency, ensuring quality, and delivering solutions that enhance business value in competitive, fast-paced technology environments."

In [14]:
model_name = "Shrav20/job-ner-deberta"  # This is trained for NER
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

print(f"Model loaded: {model_name}")
print(f"Model max length: {tokenizer.model_max_length}")
print(f"Available labels: {list(model.config.id2label.values())}")

Model loaded: Shrav20/job-ner-deberta
Model max length: 1000000000000000019884624838656
Available labels: ['O', 'B-EDUCATION', 'I-EDUCATION', 'B-SKILL', 'I-SKILL', 'B-CERTIFICATION', 'I-CERTIFICATION', 'B-LANGUAGE', 'I-LANGUAGE']


In [15]:
inputs = tokenizer(
    job,
    return_tensors="pt",
    truncation=True,
    is_split_into_words=False
)

# 2. Forward pass
with torch.no_grad():
    outputs = model(**inputs)          # outputs.logits shape: [batch, seq_len, num_labels]

# 3. Take the most probable label for each token
pred_ids = torch.argmax(outputs.logits, dim=2)[0]    # shape: [seq_len]

# 4. Convert back to words + labels
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
labels = [model.config.id2label[id_.item()] for id_ in pred_ids]

J={}

j=-1
for i, (token, label) in enumerate(zip(tokens, labels)):
    if(j>i):
        continue
    if label != "O":
        # Start a span with the current token
        span = token
        j = i + 1
        while j < len(labels) and labels[j][2:] == label[2:]:# Check consecutive tokens with the same label
            span +=" "
            span += tokenizer.convert_tokens_to_string([tokens[j]])# Concatenate subsequent tokens with the same label
            j += 1
        print(f"{span:<30} {label}")
        if label !='DATE':
            J[label[2:]] = J.get(label[2:], []) + [span]
        

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


▁engineering                   B-EDUCATION
▁project                       B-SKILL
▁management                    B-EDUCATION
▁Java                          B-SKILL
▁Python                        B-SKILL
▁SQL                           B-SKILL
▁Linux                         B-SKILL
▁engineering                   B-EDUCATION
▁Bachelor                      B-EDUCATION
▁Science                       B-EDUCATION
▁Engineering                   B-EDUCATION
▁leadership                    B-SKILL


In [16]:
jobskills = re.findall(skills_pattern, job)
print(f"Skills found: {jobskills}")

Skills found: ['Java', 'Python', 'SQL', 'Linux']


In [17]:
Job={}
if jobskills:
    Job['Skills'] = Job.get('Skills', []) + jobskills


for skill in J['SKILL']:
    if skill not in Job['Skills']:
        if skill[0]=='▁':
            skill=skill[1:]
        Job['Skills'] = Job.get('Skills', []) + [skill]

In [18]:
degrees= re.findall(d_pattern, job)
print(f"Degrees found: {degrees}")
for d in degrees:
    if "degree" in d.lower() or "bachelor" in d.lower() or "master" in d.lower() or "doctor" in d.lower() or "associate" in d.lower() or "diploma" in d.lower() or "certificate" in d.lower():
        if len(d)>60:
            Job['Degree'] = Job.get('Degree', []) + [d[:60]]
        else:
            Job['Degree'] = Job.get('Degree', []) + [d]


if 'EDUCATION' in J:
    for skill in J['EDUCATION']:
        if skill not in Job['Degree']:
            if skill[0]=='▁':
                skill=skill[1:]
            Job['Degree'] = Job.get('Degree', []) + [skill]

print(Job)

Degrees found: ['management', 'methodologies', 'mentoring', 'Bachelor ']
{'Skills': ['Java', 'Python', 'SQL', 'Linux', 'project', 'Java', 'Python', 'SQL', 'Linux', 'leadership'], 'Degree': ['Bachelor ', 'engineering', 'management', 'engineering', 'Bachelor', 'Science', 'Engineering']}


In [19]:
experience = re.findall(experience_pattern, job)
print(f"Experience found: {experience}")
if experience:
    for exp in experience:
        if int(exp[0]) < 50:  # Filter out unrealistic experience values
            Job['ExperianceYears'] = Job.get('ExperianceYears', []) + [f"{exp[0]} {exp[1]}"]  


Experience found: [('10', 'years')]


In [21]:
from custom_utils.utils import standardize_data
Job=standardize_data(Job)
L=standardize_data(L)


for key in Job:
    print(f"{key}: {Job[key]}")

print()
print("********************************")
print()
for key in L:
    print(f"{key}: {L[key]}")

Skills: ['Java', 'Python', 'Sql', 'Linux', 'Project', 'Leadership']
Degree: ['Bachelor', 'Engineering', 'Management', 'Science']
ExperianceYears: [10]

********************************

Name: ['Ram Edupuganti']
Designation: ['Principal Applications Developer', 'Senior Software Developer', 'Software Development Director', 'Senior Development Manager', 'Project Lead']
Companies worked at: ['Oracle']
Skills: ['Pl/Sql', 'Xml', 'Windows', 'Oracle ', 'Oracle Ebusiness Scm', 'Linux', 'Python', 'Development ', 'Sql', 'J2Ee', 'Java', 'Oracle Adf', 'Structured Software ', 'Html', 'Jdk', 'R']
College Name: ['Texas A&M University', 'Nagarjuna University']
Degree: ['B.S. In Electronics & Communications Engineering']


In [22]:
from custom_utils.fit_calc import total_match_score

In [24]:
print(f"Total Match Score: {total_match_score(Job, L)}")
print()
print(Job.get('Degree', []))
from custom_utils.fit_calc import skill_match_score, degree_match_score, experience_match_score
print(f"Skill Match Score: {skill_match_score(Job.get('Skills', []), L.get('Skills', []))}")
print(f"Degree Match Score: {degree_match_score(Job.get('Degree', []), L.get('Degree', []))}")
#print(f"Experience Match Score: {experience_match_score(Job.get('ExperianceYears', 0), L.get('ExperianceYears', 0))}")

Total Match Score: 0.49388431260055243

['Bachelor', 'Engineering', 'Management', 'Science']
Skill Match Score: 0.6127686252011049
Degree Match Score: 0.25


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers import AutoModel
import torch
import spacy
import spacy_transformers  # Explicitly import spacy_transformers
from pathlib import Path
from custom_utils.Process_data import process


path=Path(r"model-best")
# Load the best trained model
nlp = spacy.load(path)

model_name = "manishiitg/resume-ner"  # This is trained for NER
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

model_name2 = "Shrav20/job-ner-deberta"  # This is trained for NER
tokenizer2 = AutoTokenizer.from_pretrained(model_name)
model2 = AutoModelForTokenClassification.from_pretrained(model_name)

job = "Job Title: Software Engineer About the Role We are seeking a skilled and motivated Software Engineer to join our development team. You will be responsible for designing, developing, testing, and maintaining high-quality software solutions. The ideal candidate has a solid foundation in computer science, enjoys solving complex problems, and is eager to collaborate in a fast-paced, agile environment. Responsibilities - Design, develop, and maintain scalable and reliable software applications. - Write clean, efficient, and well-documented code. - Collaborate with product managers, designers, and other engineers to deliver new features. - Participate in code reviews, testing, and debugging. - Ensure software performance, security, and maintainability. - Contribute to continuous integration and deployment (CI/CD) processes. - Stay updated with emerging technologies and industry best practices. Requirements - Bachelor’s degree in Computer Science, Software Engineering, or related field (or equivalent experience). - Strong proficiency in one or more programming languages (e.g., Python, Java, C++, or JavaScript). - Experience with databases (SQL/NoSQL) and API development. - Familiarity with version control systems (e.g., Git). - Understanding of software development methodologies (Agile/Scrum). - Strong problem-solving and analytical skills. - Excellent communication and teamwork abilities. Preferred Qualifications - Experience with cloud platforms (AWS, Azure, or GCP). - Knowledge of DevOps tools and CI/CD pipelines. - Familiarity with front-end frameworks (React, Angular, or Vue). - Exposure to containerization technologies (Docker, Kubernetes). What We Offer - Competitive salary and benefits package. - Flexible working arrangements (remote/hybrid). - Professional development and training opportunities. - A collaborative and innovative work environment."

text="Ram Edupuganti - Resume Current: Software Development Director - Oracle Inc  - Email me on Indeed: indeed.com/r/Ram-Edupuganti/3ecdecbcba549e21  \u2022 Offering over 22 years of rich national & international experience in developing & deploying software appplicatoins development, customized solutions across Oracle Fusion HCM Global Core HR, Global Payroll, Global Absences, Compensation and all Talent modules - Performance Management, Goals Management, Career Development, Succession Management & Talent Review Meeting \u2022 Directed planning, strategy development and implementation & business solution delivery with recent successful implantations at Schneider Electric, Macy's, Pike International, British Telcom. \u2022 Accomplished in relational data modeling, data warehouse design and implementation, object oriented design and development, developing transactional & business analytics in CRM, SCM and HCM domains from client server to cloud SaaS. \u2022 Led large-scale business application architecture and design efforts; developed and maintained solutions for various business functional areas; assisted in resolving integration and interface issues between various systems with focus on optimizing performance and scalability \u2022 Extensive experience in all aspects of project management including budgeting and cost optimization, risk assessments and control, technical feasibility studies, project scope definition, estimations & cost control & so on \u2022 Recruit, Develop and led high-performing teams with high motivation and enhanced cross- functional collaboration  WORK EXPERIENCE  Software Development Director  Oracle Inc -  April 2015 to Present  Employer: Oracle Inc -  January 1998 to Present  98 with Oracle Inc. Growth Path & Deputations:  Role: Senior Development Manager  Employer: Oracle Inc -  December 2011 to March 2015  Senior Development Manager  Employer: Oracle Inc -  March 2010 to November 2011  https://www.indeed.com/r/Ram-Edupuganti/3ecdecbcba549e21?isid=rex-download&ikw=download-top&co=IN   Role: Principal Applications Developer  Oracle Inc -  September 2004 to February 2010  Project Lead  Oracle Inc -  September 1999 to August 2004  Senior Software Developer  Oracle Inc, Inc. -  January 1998 to August 1999  R-Systems -  Sacramento, CA -  January 1997 to January 1998  Key Result Areas: \u2022 Providing overall leadership to the entire project team, mapping clients' requirements, transforming requirements into stipulations and providing them best solutions within the scope of project requirements \u2022 Creating and managing the estimates, project plan, project schedule, resource allocation and expenses; monitoring and reporting on standards & performance targets \u2022 Ensuring the delivery of quality releases on schedules using continuous improvement initiatives; adhering to quality norms throughout the project implementation \u2022 Working on service strategy, transition, operations, process improvement, process management, team building, training, hiring and client relationship management \u2022 Identifying and implementing strategies for building team effectiveness by promoting a spirit of cooperation among the team members \u2022 Supporting continuous improvement by investigating alternatives and new technologies and presenting the same for architectural review \u2022 Liaising with stakeholders during the course of problem diagnoses, requirements gathering, detailed level design, development, system test and production implementation to ensure that optimal resolutions are achieved  KNOWLEDGE PURVIEW \u2022 Machine Learning Algorithms supervised, unsupervised, NLP, chat bot and deep learning with Python \u2022 Building transactional applications & analytical solutions in SaaS model \u2022 Oracle fusion role based data security set up and customization for complex security requirements \u2022 Reporting Layer (Subject Areas) for fusion HCM cloud for all HCM modules \u2022 Data Warehousing design methodologies, star and snowflake schema designs, aggregations \u2022 OBIEE, Data Visualization Desktop, BICS, DVCS, Oracle Analytics Cloud (OAC), Tableau \u2022 Design and build dashboards, KPIs using Oracle BI platform and OTBI, Standard Reports using BI Publisher \u2022 Design and build mappings, knowledge modules, load plans using Oracle Data Integrator (ODI)    \u2022 Relational data modeling, Object Oriented Modeling & Design, UML, SOAP \u2022 Oracle 10g, 11g, 12c \u2022 SQL, PL/SQL, HTML, XML, JAVA, JDK, J2EE, Oracle ADF, Oracle JDeveloper \u2022 Applications Development for Oracle CRM, Oracle ebusiness SCM, \u2022 Applications/database/SQL/Batch program performance tuning \u2022 UNIX, Windows, Linux  R-Systems -  April 1996 to January 1998  Growth Path & Deputations:  Department of Corrections -  Sacramento, CA -  February 1996 to January 1997  Secretary of State  Salem, Tamil Nadu -  December 1995 to February 1996  Oracle Corporation -  Redwood Shores, CA -  January 1995 to December 1995  EDUCATION  M.S. in Computer Engineering with honors  Texas A&M University -  Kingsville, TX  1994  B.S. in Electronics & Communications Engineering with honors  Nagarjuna University  1992  SKILLS  SOFTWARE DEVELOPMENT (3 years), STRUCTURED SOFTWARE (3 years), BUSINESS INTELLIGENCE (1 year), ORACLE (1 year), ARCHITECTURE (Less than 1 year)  ADDITIONAL INFORMATION  CORE COMPETENCIES  Software Development & Consulting  Oracle Fusion Applications Architecture and Functionality    Oracle Business Intelligence development and implementations  Technology Planning  Delivery Management  Client Engagements (Stakeholders/Business)  Continuous Process Enhancement  Agile/Scrum Methodologies  Project Management"

ram,job=process(nlp, model, tokenizer, model2, tokenizer2, text,job)


Resume Skills found: ['Linux', 'SQL', 'STRUCTURED SOFTWARE (3 years)', 'XML', 'DEVELOPMENT (3 years)', 'R', 'Python', 'HTML', 'Oracle ebusiness SCM', 'Oracle ADF', 'Windows', 'ORACLE (1 year)', 'PL/SQL', 'J2EE', 'JAVA', 'JDK']

********************************

Total Match Score: 0.38516315075820573
Skills: ['Python', 'Java', 'Javascript', 'Sql', 'Nosql', 'Git', 'Aws', 'Azure', 'Gcp', 'React', 'Angular', 'Vue', 'Docker', 'Kubernetes']
Degree: ["Bachelor'S Degree In Computer Science, Software Engineering,"]

********************************

Name: ['Ram Edupuganti']
Designation: ['Software Development Director', 'Senior Development Manager', 'Project Lead', 'Principal Applications Developer', 'Senior Software Developer']
Companies worked at: ['Oracle']
Skills: ['Linux', 'Sql', 'Structured Software ', 'Xml', 'Development ', 'R', 'Python', 'Html', 'Oracle Ebusiness Scm', 'Oracle Adf', 'Windows', 'Oracle ', 'Pl/Sql', 'J2Ee', 'Java', 'Jdk']
College Name: ['Nagarjuna University', 'Texas A&M 

In [20]:
from custom_utils.encoding_similarity import document_level_similarity,skill_level_similarity

doc_sim=document_level_similarity(ram,job)
skill_sim=skill_level_similarity(ram.get('Skills',[]),job.get('Skills',[]))
print(f"Document Level Similarity: {doc_sim}")
print(f"Skill Level Similarity: {skill_sim.mean()}")


Document-level similarity: 0.4168
Skill-level similarity matrix:
[[ 0.3870465   0.42135003  0.2760898   0.24215923  0.24107306  0.3003102
   0.32965192  0.15218383  0.23381597  0.20673661  0.24064614  0.22314179
   0.26573172  0.2014278 ]
 [ 0.29998437  0.3222519   0.17510812  1.          0.38845807  0.17878534
   0.23115478  0.23976679  0.1393481   0.03065043  0.12513937  0.03703839
   0.10393425  0.03252728]
 [ 0.4303702   0.24462086  0.16206285  0.12683252  0.09462208  0.22113992
   0.1330415   0.13043803  0.13259882  0.16276078  0.00562657  0.22788824
   0.16158019  0.12207113]
 [ 0.20798826  0.23092109  0.14498527  0.1847328   0.11292502  0.27131695
   0.16350558  0.11927973  0.14678678  0.1288253   0.08141003  0.12283674
   0.13748387  0.16710015]
 [ 0.46478397  0.15921232  0.09362569  0.20523998  0.20962103  0.16502838
   0.38307005  0.17842317  0.11471272  0.15293238  0.07894427  0.1777972
   0.18062203  0.23521316]
 [ 0.264049    0.30470252  0.40247062  0.26430967  0.13108289 

In [8]:
job["Skills"]

['Python',
 'Java',
 'Javascript',
 'Sql',
 'Nosql',
 'Git',
 'Aws',
 'Azure',
 'Gcp',
 'React',
 'Angular',
 'Vue',
 'Docker',
 'Kubernetes']

In [ ]:
from custom_utils.utils import extract_text_from_pdf


text=extract_text_from_pdf(r"C:\Users\yby39\Downloads\Yassine_Benyahia_CV.pdf.pdf.pdf")

ImportError: cannot import name 'total_match_score' from 'custom_utils.utils' (c:\ML\Resume-assistant\custom_utils\utils.py)

In [10]:
text

'Yassine Benyahia Artificial Intelligence & Machine Learning, Technology for Sustainable Development. Self-learning & Research: Continuous learning in AI, ML through online platforms. Email: yassinebnyahia@gmail.com | GitHub: github.com/YassineBen-Yahia | LinkedIn: https://www.linkedin.com/in/yassine-benyahia-523887353 Institut National des Sciences Appliquées et de Technologie (INSAT) — Tunis, Tunisia Third-year student specializing in Software Engineering with a strong foundation in Machine Learning and Deep Learning, as well as solid knowledge of Object-Oriented Programming (Java), Databases (SQL), Operating Systems (Linux), Computer Architecture, and Statistical Methods. An AI-powered Resume Assistant : An intelligent AI assistant that analyzes your CV and job descriptions using NLP and NER to evaluate compatibility based on skills, education, and experience. It acts as a smart career advisor, providing instant feedback on your job fit and improvement suggestions. Parking Lot Detec

In [19]:
ram,job=process(nlp, model, tokenizer, model2, tokenizer2, text,job)

Resume Skills found: ['Linux', 'SQL', 'PyTorch', 'Projects & Academic Work', 'Pandas', 'CSS', 'Git', 'HTML', 'Scikit-learn', 'Python', 'Java']

********************************

Total Match Score: 0.3837306091294769
Skills: ['Python', 'Java', 'Javascript', 'Sql', 'Nosql', 'Git', 'Aws', 'Azure', 'Gcp', 'React', 'Angular', 'Vue', 'Docker', 'Kubernetes']
Degree: ["Bachelor'S Degree In Computer Science, Software Engineering,"]

********************************

Name: ['Yassine Benyahia']
College Name: ['Institut National Des Sciences Appliques Et De Technologie (Insat)']
Skills: ['Linux', 'Sql', 'Pytorch', 'Projects & Academic Work', 'Pandas', 'Css', 'Git', 'Html', 'Scikit-Learn', 'Python', 'Java']
Email Address: ['Yassinebnyahia@Gmail.Com']


In [18]:

job = "Job Title: Software Engineer About the Role We are seeking a skilled and motivated Software Engineer to join our development team. You will be responsible for designing, developing, testing, and maintaining high-quality software solutions. The ideal candidate has a solid foundation in computer science, enjoys solving complex problems, and is eager to collaborate in a fast-paced, agile environment. Responsibilities - Design, develop, and maintain scalable and reliable software applications. - Write clean, efficient, and well-documented code. - Collaborate with product managers, designers, and other engineers to deliver new features. - Participate in code reviews, testing, and debugging. - Ensure software performance, security, and maintainability. - Contribute to continuous integration and deployment (CI/CD) processes. - Stay updated with emerging technologies and industry best practices. Requirements - Bachelor’s degree in Computer Science, Software Engineering, or related field (or equivalent experience). - Strong proficiency in one or more programming languages (e.g., Python, Java, C++, or JavaScript). - Experience with databases (SQL/NoSQL) and API development. - Familiarity with version control systems (e.g., Git). - Understanding of software development methodologies (Agile/Scrum). - Strong problem-solving and analytical skills. - Excellent communication and teamwork abilities. Preferred Qualifications - Experience with cloud platforms (AWS, Azure, or GCP). - Knowledge of DevOps tools and CI/CD pipelines. - Familiarity with front-end frameworks (React, Angular, or Vue). - Exposure to containerization technologies (Docker, Kubernetes). What We Offer - Competitive salary and benefits package. - Flexible working arrangements (remote/hybrid). - Professional development and training opportunities. - A collaborative and innovative work environment."


In [16]:
from custom_utils.fit_calc import total_match_score
print(f"Total Match Score: {total_match_score(job, ram)}")

print()
print(job.get('Degree', []))
from custom_utils.fit_calc import skill_match_score, degree_match_score, experience_match_score
print(f"Skill Match Score: {skill_match_score(job.get('Skills', []), ram.get('Skills', []))}")
print(f"Degree Match Score: {degree_match_score(job.get('Degree', []), ram.get('Degree', []))}")
#print(f"Experience Match Score: {experience_match_score(job.get('ExperianceYears', 0), L.get('ExperianceYears', 0))}")

Total Match Score: 0.3837306091294769

["Bachelor'S Degree In Computer Science, Software Engineering,"]
Skill Match Score: 0.26746121825895375
Degree Match Score: 0.0
